In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler, PolynomialFeatures, StandardScaler, MinMaxScaler


In [3]:
data = pd.read_csv("cleaned_input/train_consolidated.csv")

In [4]:
data.head()
data = data.drop(["msno", "Unnamed: 0"], axis=1)
data.shape

(504818, 122)

In [5]:
model_data = data.sample(frac=.1)
X_train = model_data.drop(['is_churn'], axis=1);
y_train = model_data['is_churn'];
X_train.isnull().any()

trans_count                    False
date_min                       False
date_max                       False
date_count                     False
num_25_sum                     False
num_50_sum                     False
num_75_sum                     False
num_985_sum                    False
num_100_sum                    False
num_unq_sum                    False
total_secs_sum                 False
city                           False
bd                             False
registered_via                 False
registration_init_year         False
registration_init_month        False
registration_init_date         False
payment_method_id              False
payment_plan_days              False
plan_list_price                False
actual_amount_paid             False
is_auto_renew                  False
is_cancel                      False
discount                       False
is_discount                    False
amt_per_day                    False
membership_duration            False
m

In [6]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train, scoring="neg_log_loss", train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [7]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}


In [18]:
lgbm_pipeline = Pipeline([('lgbm', LGBMClassifier())])

parameters = {
    'learning_rate': [0.005],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

CV = GridSearchCV(lgbm_pipeline, parameters, scoring = 'neg_log_loss', n_jobs= 3)
CV.fit(X_train, y_train)   

print('Best score and parameter combination = ')

print(CV.best_score_)    
print(CV.best_params_)    


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/James/anaconda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/James/anaconda/lib/pyth

In [ ]:
   params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 3, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# Create parameters to search
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt', 
          objective = 'binary', 
          n_jobs = 5, # Updated from 'nthread' 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'])

# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=1, cv=4, n_jobs=-1)
# Run the grid
grid.fit(allTrainData, allTrainLabels)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)